In [ ]:
import duckdb
con = duckdb.connect('database/analytics.duckdb')

# 1. ¿Se crearon las columnas compactas?
df_check = con.execute("SELECT * FROM silver.canales LIMIT 5").df()
print(df_check.columns) 

# 2. ¿El índice de digitalización tiene sentido? (Debe ser entre 0 y 1)
stats = con.execute("SELECT MIN(index_digitalizacion_ult6), MAX(index_digitalizacion_ult6) FROM silver.canales").fetchone()
print(f"Rango Digitalización: {stats}")